# Automatización de la Matriz de Requerimientos para el Plan Anual de Capacitación
Esta libreta automatiza la lectura de archivos de evaluación de desempeño y genera una matriz que indica qué trabajadores necesitan capacitación en qué temas.

## <b>Importación de librerías </b>>

Importamos las librerías necesarias para manipular archivos Excel, manejar rutas y procesar los datos. En total, se subierón 55 archivos.

In [157]:
import pandas as pd 
import glob
import numpy as np
import os
# Ruta de la carpeta donde están los archivos csv
carpeta = r"C:\Users\mflor\Downloads\DatosNuevos"
# Obtener la lista de archivos csv en la carpeta
archivos= glob.glob(os.path.join(carpeta,"**", "*.csv"), recursive=True)


In [159]:
print(len(archivos))

55


En la siguiente línea de código se muestran los primeros 10 archivos de los 55 que se procesaran.

In [166]:
archivos[:10]

['C:\\Users\\mflor\\Downloads\\DatosNuevos\\Area1.csv',
 'C:\\Users\\mflor\\Downloads\\DatosNuevos\\Area10.csv',
 'C:\\Users\\mflor\\Downloads\\DatosNuevos\\Area11.csv',
 'C:\\Users\\mflor\\Downloads\\DatosNuevos\\Area12.csv',
 'C:\\Users\\mflor\\Downloads\\DatosNuevos\\Area13.csv',
 'C:\\Users\\mflor\\Downloads\\DatosNuevos\\Area14.csv',
 'C:\\Users\\mflor\\Downloads\\DatosNuevos\\Area15.csv',
 'C:\\Users\\mflor\\Downloads\\DatosNuevos\\Area16.csv',
 'C:\\Users\\mflor\\Downloads\\DatosNuevos\\Area17.csv',
 'C:\\Users\\mflor\\Downloads\\DatosNuevos\\Area18.csv']

Los archivos cargados se guardan en tablas (dataframes). A continuación, se muestra el formato de los archivos. 

La primera columna contiene los temas, 
en el encabezado se encuentra la nomina del trabajador y al lado de las nominas, la modalidad del tema. Para saber si el tema necesita ser cubierto, nos interesa la tercera columna correspondiente a la nomina del trabajador. Esta contiene claves (0,1,2,3)

- 3: Indica urgencia ALTA- Agregar al PAC  actual y programar entrenamiento  en  los primeros 4 meses
- 2 : Indica urgencia MEDIA- Agregar al PAC  y programar en los primeros 8 meses (COMPETENCIAS DE DESARROLLO)
- 1: Indica urgencia BAJA- Agregar al PAC y Programar dentro de los 12 meses 
- 0: CUBIERTO- Competencia cubierta 

(PAC significa Plan Anual de Capacitación)

### Formato de los archivos cargados
![Archivo Limpio](imagenes/DNCPROCESADA.png)

In [170]:
#lista de dataframes
dataframes = [pd.read_csv(archivo,encoding='utf-8',low_memory=False) for archivo in archivos]

## <b>Limpieza de los datos </b>

Antes de generar la matriz de requerimientos de capacitación, es fundamental asegurarse de que los archivos de entrada tengan el formato adecuado para evitar errores en el procesamiento.

Primero, se verifica que existan las columnas clave: taller y curso, ya que contienen la información necesaria para identificar si un trabajador asistió a una capacitación.

Después, se realiza una conversión de celdas vacías. Es común que algunos archivos contengan valores nulos (NaN). Para prevenir errores durante el análisis, estos valores se reemplazan por cadenas vacías (""), lo cual permite trabajar con los datos de manera más consistente.



In [174]:
#Lidiar con valores nulos
i=1
for tabla in dataframes:
    # Verificar si las columnas 'Curso' y 'Taller' existen antes de hacer la conversión
    if 'Curso' in tabla.columns:
        tabla["Curso"] = tabla["Curso"].fillna("").astype(str)
    else:
        print("Columna 'Curso' no encontrada en el DataFrame %i"%i)
        print("La ubicación del archivo es %s"%archivos[i-1])

    if 'Taller' in tabla.columns:
        tabla["Taller"] = tabla["Taller"].fillna("").astype(str)
    else:
        print("Columna 'Taller' no encontrada en el DataFrame")
    i+=1

## Ejemplo 

En la siguiente línea del código se muestra el formato de los 55 archivos analizados. En el encabezado se muestra el número de nomina, mientras que del lado izquierdo se muestra el tema. Esta tabla evalua el rendimiento del  trabajor.


In [178]:
dataframes[0]

,Tema,761268.0,761268.1,761268.2,776169.0,776169.1,776169.2,769466.0,769466.1,769466.2,773190.0,773190.1,773190.2,774233.0,774233.1,774233.2,Curso,Taller
0,Computo estadístico,3,1,2,3,1,2,3,1,2,3,1,2,3,1,2,,
1,Procesamiento_Lenguaje_Natural,3,1,2,3,1,2,3,1,2,3,1,2,3,1,2,,
2,Ciberseguridad_Avanzada,3,1,2,3,1,2,3,1,2,3,1,2,3,1,2,,
3,Bases_de_Datos_SQL,2,1,1,2,1,1,3,1,2,3,1,2,2,1,1,,
4,Bases_de_Datos_SQL,3,1,2,3,1,2,3,1,2,3,1,2,3,1,2,,
5,Computer_Vision,3,2,1,3,2,1,3,2,1,3,2,1,3,2,1,,
6,conocimiento de requisitos especificos del cli...,3,1,2,3,1,2,3,1,2,3,1,2,3,1,2,,
7,competencia en auditoria dtld,2,1,1,2,1,1,3,1,2,3,1,2,2,1,1,,
8,CI_CD_Pipelines,3,2,1,3,2,1,3,2,1,3,2,1,3,2,1,,
9,Data_Engineering,3,1,2,3,1,2,3,1,2,3,1,2,3,1,2,,


### <b>Agrupación de temas por equivalencia de contenido<b>

Se identificó que ciertos cursos o temas, aunque tienen nombres distintos, abordan los mismos contenidos o competencias. Por lo tanto, se agruparon bajo una categoría común para evitar duplicidad y facilitar la interpretación de los datos.

In [183]:
diccionario_temas = {
    "Metodologias_Agiles_Kanban": "Curso de Kanban y metodologías ágiles",
    "Automatizacion_Sistemas": "Automatización de procesos con Python y RPA",
    "Ciberseguridad_Basica": ["Fundamentos de ciberseguridad", "Protección de datos básicos"],
    "Ciberseguridad_Avanzada": "Gestión avanzada de seguridad informática",
    "Desarrollo_Web_Frontend": "HTML, CSS, JavaScript, frameworks como React",
    "Desarrollo_Web_Backend": "Node.js, Django, bases de datos, API REST",
    "Bases_de_Datos_SQL": "MySQL, PostgreSQL, consultas SQL",
    "Bases_de_Datos_NoSQL": "MongoDB, Redis, bases de datos orientadas a documentos",
    "DevOps_Practicas": ["CI/CD", "Docker", "Kubernetes", "Automatización de despliegues"],
    "Cloud_Computing": ["AWS", "Azure", "Google Cloud", "Servicios en la nube"],
    "Machine_Learning_Basico": ["Regresión lineal", "Clasificación", "Preprocesamiento de datos"],
    "Machine_Learning_Avanzado": "Redes neuronales, árboles de decisión, SVM",
    "Inteligencia_Artificial": "Agentes inteligentes, razonamiento automatizado",
    "Big_Data": "Hadoop, Spark, procesamiento de grandes volúmenes de datos",
    "Programacion_Python": ["Python avanzado", "Librerías NumPy, pandas, matplotlib"],
    "Programacion_Java": "Java SE, programación orientada a objetos",
    "Control_Versiones": ["Git", "GitHub", "Branching y merge"],
    "Testing_Automatizado": "Pruebas unitarias y automatización con Selenium",
    "Arquitectura_Software": "Patrones de diseño, arquitectura en capas",
    "UI_UX_Design": "Principios de diseño de interfaces y experiencia de usuario",
    "Internet_de_las_Cosas": "Sensores, microcontroladores y comunicación IoT",
    "Blockchain": ["Criptomonedas", "Smart contracts", "Blockchain básico"],
    "Analisis_Datos": ["Análisis exploratorio de datos", "Visualización con Python"],
    "Inteligencia_Business": "Power BI, Tableau, dashboards interactivos",
    "Automatizacion_RPA": "Automatización de tareas repetitivas con UiPath o Automation Anywhere",
    "Realidad_Aumentada_VR": "Unity3D, desarrollo de aplicaciones AR/VR",
    "Metodologias_DevOps": "Scrum, Kanban, integración continua",
    "Seguridad_Cloud": "Seguridad en AWS, Azure y Google Cloud",
    "Gestion_Proyectos_IT": "PMI, gestión de proyectos de TI",
    "Docker": "Creación de contenedores y despliegue de aplicaciones",
    "Kubernetes": ["Orquestación de contenedores", "Clústeres y pods"],
    "Serverless_Computing": "Funciones sin servidor en AWS Lambda o Azure Functions",
    "Microservicios": "Diseño e implementación de microservicios",
    "APIs_REST": "Diseño y consumo de APIs RESTful",
    "GraphQL": "Consulta y manipulación de datos con GraphQL",
    "CI_CD": "Integración y despliegue continuo",
    "IaC": "Terraform, Ansible y automatización de infraestructura",
    "Monitoreo": "Prometheus, Grafana, monitoreo de sistemas",
    "Testing_Unitario": ["pytest", "JUnit", "Pruebas automatizadas"],
    "Testing_Integracion": "Pruebas de integración de software",
    "Testing_Carga": "Pruebas de rendimiento y carga con JMeter",
    "Python_Data_Science": "Pandas, NumPy, scikit-learn",
    "R_Data_Science": "R, ggplot2, tidyverse",
    "AI_Machine_Learning": "Modelos predictivos y aprendizaje supervisado",
    "Deep_Learning": "Redes neuronales profundas, TensorFlow, PyTorch",
    "Procesamiento_Lenguaje_Natural": ["NLTK", "spaCy", "Análisis de texto"],
    "Computer_Vision": "OpenCV, detección de objetos, visión por computadora",
    "Big_Data_Tools": "Hadoop, Spark, procesamiento distribuido",
    "ETL_Procesos": ["Extracción, Transformación y Carga de datos", "Airflow"],
    "Data_Warehousing": ["Diseño de almacenes de datos", "ETL y reporting"],
    "BI_Dashboarding": ["Tableau", "Power BI", "Dashboards interactivos"],
    "Analitica_Predictiva": "Regresión, clasificación, modelos de predicción",
    "IoT_Cloud": "AWS IoT, Azure IoT Hub, conectividad de dispositivos",
    "Blockchain_Enterprise": ["Ethereum", "Hyperledger", "Smart contracts empresariales"],
    "Smart_Contracts": "Desarrollo y auditoría de contratos inteligentes",
    "Fintech_Tecnologias": ["Pagos digitales", "Blockchain en finanzas", "RegTech"],
    "VR_AR_MR": ["Unity", "Unreal Engine", "Experiencias inmersivas"],
    "Gamificacion": ["Diseño de juegos", "Motivación y engagement en software"],
    "UX_Research": ["Estudios de usuario", "Pruebas de usabilidad"],
    "Data_Engineering": ["ETL", "Modelado de datos", "Pipeline de datos"],
    "Cloud_Security": ["Seguridad en AWS, Azure, Google Cloud"],
    "Server_Maintenance": ["Mantenimiento de servidores", "Monitoreo y backups"],
    "Networking": ["TCP/IP", "Redes LAN/WAN", "Protocolos de comunicación"],
    "Virtualization": ["VMware", "Hyper-V", "Virtualización de servidores"],
    "Containers_Advanced": ["Docker avanzado", "Orquestación de contenedores"],
    "AI_Ethics": "Ética en IA y sesgos algorítmicos",
    "Quantum_Computing": ["Qiskit", "Principios de computación cuántica"],
    "Edge_Computing": "Procesamiento en el borde, IoT",
    "Robotic_Process_Automation": "RPA con UiPath, Automation Anywhere",
    "Low_Code_Platforms": "Plataformas low-code como Mendix o PowerApps",
    "No_Code_Solutions": "Automatización sin código",
    "Chatbots": "Diseño de chatbots con Dialogflow o Rasa",
    "Voice_Assistants": "Alexa Skills, Google Assistant",
    "Data_Visualization_Advanced": "Dash, Plotly, visualizaciones interactivas",
    "Data_Analytics_Python": ["Análisis de datos con Python", "Pandas, matplotlib"],
    "Data_Analytics_R": "Análisis estadístico con R",
    "Serverless_Architecture": "Diseño de aplicaciones serverless",
    "Event_Driven_Architecture": "Arquitecturas basadas en eventos",
    "Software_Architecture": "Patrones de diseño y arquitectura empresarial",
    "Code_Review": "Buenas prácticas de revisión de código",
    "Agile_Testing": ["Testing en entornos ágiles", "Pruebas continuas"],
    "API_Design": "Diseño de APIs RESTful y documentación",
    "CI_CD_Pipelines": ["Jenkins", "GitLab CI/CD", "Pipeline automation"],
    "Security_Best_Practices": "Buenas prácticas de seguridad informática",
    "Cloud_Native": ["Desarrollo cloud-native", "Microservicios y contenedores"],
    "Incident_Management": "Gestión de incidentes y resolución rápida",
    "SRE_Best_Practices": "Principios de Site Reliability Engineering",
    "Performance_Optimization": "Optimización de aplicaciones y base de datos",
    "Software_Maintenance": "Mantenimiento y actualización de software",
    "Configuration_Management": ["Ansible", "Puppet", "Chef"],
    "Observability": ["Logging", "Metrics", "Tracing"],
    "Business_Analytics": ["KPIs empresariales", "Análisis de negocio"],
    "Predictive_Modeling": "Modelos predictivos y forecasting",
    "Reinforcement_Learning": ["Q-Learning", "Deep Q-Networks"],
    "Graph_Analytics": "Análisis de redes y grafos",
    "Recommendation_Systems": ["Filtrado colaborativo", "Sistemas de recomendación"],
    "Natural_Language_Processing": ["Procesamiento de texto y NLP"],
    "Speech_Recognition": ["Reconocimiento de voz y audio"],
    "Image_Processing": "Procesamiento de imágenes con OpenCV",
    "Computer_Vision_Advanced": "Detección de objetos y visión artificial",
    "Digital_Twin": "Modelos digitales de sistemas físicos",
    "Industrial_IoT": "Sensores y automatización industrial",
    "Robotics_Programming": "Programación de robots industriales",
    "VR_Game_Design": ["Diseño de juegos VR", "Unity3D para VR"],
    "AR_Mobile_Apps": "Desarrollo de apps AR móviles",
    "Cloud_Data_Lakes": "Almacenamiento y gestión de datos en la nube",
    "Streaming_Data": "Procesamiento de datos en tiempo real",
    "Data_Governance": "Políticas y estándares de datos",
    "Master_Data_Management": "Gestión de datos maestros",
    "Identity_Access_Management": "Control de acceso y autenticación",
    "Zero_Trust_Security": "Implementación de Zero Trust",
    "Penetration_Testing": "Pruebas de penetración y hacking ético",
    "Incident_Response": ["Plan de respuesta a incidentes", "Gestión de crisis"],
    "Container_Security": "Seguridad de contenedores",
    "Cloud_Migration": ["Migración a la nube", "Planificación de transición"],
    "API_Security": "Seguridad de APIs y autenticación",
    "Edge_AI": ["IA en dispositivos edge", "Procesamiento local"],
    "Quantum_Machine_Learning": "ML en computación cuántica",
    "AI_Explainability": "Explicabilidad de modelos de IA",
    "Data_Privacy": ["Privacidad de datos", "GDPR y regulaciones"],
    "Autonomous_Vehicles_Tech": "Tecnologías de vehículos autónomos",
    "Smart_Cities_Tech": "IoT y sistemas urbanos inteligentes",
    "Industry_4_0": "Automatización y manufactura inteligente",
    "Edge_Cloud_Hybrid": "Arquitecturas híbridas nube-edge",
    "AI_Ops": "Operaciones de IA y optimización",
    "Digital_Transformation": "Transformación digital de negocios",
    "Edge_Analytics": "Análisis de datos en el edge",
    "Federated_Learning": ["Aprendizaje federado", "Colaboración sin compartir datos"],
    "MLOps": "Prácticas de ML en producción",
    "TinyML": ["ML en microcontroladores", "Modelos ligeros de ML"],
    "AI_Ethics_Principles": "Principios éticos en IA",
    "5G_Technology": ["Redes 5G", "Telecomunicaciones avanzadas"],
    "Blockchain_Finance": ["Blockchain en finanzas", "Smart contracts financieros"],
    "CyberThreat_Intelligence": "Inteligencia de amenazas cibernéticas",
    "Serverless_Security": "Seguridad en aplicaciones serverless",
    "Digital_Twin_Industrial": "Gemelos digitales en la industria",
    "AR_Collaboration_Tools": "Herramientas de colaboración AR",
    "Quantum_Simulation": "Simulación cuántica y modelado",
    "Robotic_Process_Automation_Advanced": ["RPA avanzado", "Automatización industrial"],
    "Container_Orchestration": "Orquestación de contenedores",
    "API_Gateway_Management": "Gestión de gateways de API",
    "Data_Observability": "Monitoreo y visibilidad de datos",
    "Predictive_Maintenance_IIoT": "Mantenimiento predictivo con IIoT",
    "Graph_AI": "Analítica de grafos con IA",
    "Conversational_AI": "Chatbots y asistentes conversacionales",
    "Explainable_AI_Techniques": "Técnicas de explicabilidad en IA",
    "Cloud_Cost_Optimization": "Optimización de costos en la nube",
    "Zero_Trust_Architecture": "Arquitectura Zero Trust",
    "Privacy_Preserving_AI": "IA preservando privacidad de datos"
}
len(diccionario_temas)

149

La siguiente función encuentra la clave correspondiente que agrupara los temas, utilizando el diccionario anterior.

In [186]:
from unidecode import unidecode  # Para eliminar acentos

In [188]:
def encontrar_nuevo_indice(texto):
    texto = unidecode(str(texto).strip().lower())  # Asegurar que el valor es string y normalizar
    for clave, valores in diccionario_temas.items():
        if isinstance(valores, list):  # Si hay una lista de sinónimos
            for valor in valores:
                valor_normalizado = unidecode(valor.strip().lower())  # Normalizar cada valor
                if valor_normalizado in texto:
                    return clave
        else:  # Si es un único valor
            valor_normalizado = unidecode(valores.strip().lower())  # Normalizar el valor único
            if valor_normalizado in texto:
                return clave
    return texto  # Si no hay coincidencia, se deja el mismo índice

## <b>Agrupar los temas de cada archivo cargado<b>

Utilizando la función anterior los temas de cada archivo se reemplaza por la clave correspondiente.

In [192]:

for tabla in dataframes:
    temas=list(tabla.Tema)
    tabla.Tema = tabla.Tema.map(encontrar_nuevo_indice)
    #print(tabla)

## <b>Cargar la tabla de Requerimientos y Plan Anual de Capacitacion<b>

Esta tabla contiene como encabezados las claves o nombres de los temas de capacitación y utiliza como índice la nómina de los trabajadores.
El objetivo es identificar, para cada trabajador, si necesita revisar o capacitarse en un tema específico.
Para ello, si un trabajador requiere capacitación en un tema, la celda correspondiente se marcará con una "x"; en caso contrario, la celda se deja en blanco (o NaN).

In [195]:
Principal=pd.read_csv(r'C:\Users\mflor\Downloads\Principal2.csv',index_col=0,encoding='utf-8')
Principal

,metodologias_agiles_kanban,automatizacion_sistemas,ciberseguridad_basica,ciberseguridad_avanzada,desarrollo_web_frontend,desarrollo_web_backend,bases_de_datos_sql,bases_de_datos_nosql,devops_practicas,cloud_computing,...,container_orchestration,api_gateway_management,data_observability,predictive_maintenance_iiot,graph_ai,conversational_ai,explainable_ai_techniques,cloud_cost_optimization,zero_trust_architecture,privacy_preserving_ai
nominas,,,,,,,,,,,,,,,,,,,,,
776532,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
770479,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
763354,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
775239,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
861202,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
762375,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
761244,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
772870,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## <b>Buscar las nominas de los trabajadores que tienen una necesidad de capacitación en uno de los temas que son curso o  taller<b>

En este bloque se procesa cada uno de los 55 archivos cargados (dataframes) para identificar los temas (cursos o talleres) en los que cada trabajador requiere capacitación.

In [199]:

nominas=[]
for tabla in dataframes:
    #Se determina la posición de la columna llamada "Curso" para usarla como referencia en la tabla.
    columna_curso_idx = tabla.columns.get_loc("Curso")
    columna_curso_idx
    """Filtrar filas con necesidad de capacitación:
       Se seleccionan las filas donde en las columnas "Curso" o "Taller" aparece una "x" (sin importar mayúsculas o minúsculas), 
       lo que indica que el tema es un curso o taller activo para el trabajador."""
    indices_x = list(tabla[(tabla["Curso"].str.lower()== "x") | (tabla["Taller"].str.lower() == "x")].index)
    indices_x
    """ Ademas se busca si un trabajador tiene un numero distinto de 0 en la tercer columna correspondiente a su nomina. Si esto sucede 
        el trabajador requiere capacitación."""
    for indice in indices_x: 
            for i in range(3, columna_curso_idx, 3):#columna
                # Obtener el valor de la tercera columna 
                valor = tabla.iloc[indice,i]
                # Si el valor obtenido es diferente de 0 y no vacio indica que el trabajador requiere capacitación. 
                #En este caso, se crea una lista con pares [Tema, Nómina] que vinculan al trabajador con el tema que debe revisar.
                if isinstance(valor, (int, float, np.number)) and valor != 0 and pd.notnull(valor):
                    nominas.append([tabla.Tema[indice], float(tabla.columns[i])])

In [201]:
len(nominas)

2785

In [211]:
nominas[:15]

[['procesamiento_lenguaje_natural', 761825],
 ['procesamiento_lenguaje_natural', 767681],
 ['procesamiento_lenguaje_natural', 770437],
 ['procesamiento_lenguaje_natural', 769424],
 ['procesamiento_lenguaje_natural', 764646],
 ['procesamiento_lenguaje_natural', 776263],
 ['procesamiento_lenguaje_natural', 772250],
 ['procesamiento_lenguaje_natural', 776324],
 ['machine_learning_basico', 761825],
 ['machine_learning_basico', 767681],
 ['machine_learning_basico', 770437],
 ['machine_learning_basico', 769424],
 ['machine_learning_basico', 764646],
 ['machine_learning_basico', 776263],
 ['machine_learning_basico', 772250]]

In [207]:
# Convertir los valores flotantes a enteros
nominas = [[nombre, int(valor)] for nombre, valor in nominas]

In [213]:
nominas[:15]

[['procesamiento_lenguaje_natural', 761825],
 ['procesamiento_lenguaje_natural', 767681],
 ['procesamiento_lenguaje_natural', 770437],
 ['procesamiento_lenguaje_natural', 769424],
 ['procesamiento_lenguaje_natural', 764646],
 ['procesamiento_lenguaje_natural', 776263],
 ['procesamiento_lenguaje_natural', 772250],
 ['procesamiento_lenguaje_natural', 776324],
 ['machine_learning_basico', 761825],
 ['machine_learning_basico', 767681],
 ['machine_learning_basico', 770437],
 ['machine_learning_basico', 769424],
 ['machine_learning_basico', 764646],
 ['machine_learning_basico', 776263],
 ['machine_learning_basico', 772250]]

In [215]:
"""Finalmente, se llena la Matriz de Requerimientos y Plan Anual de Capacitación (Principal), que contiene los temas (columnas) y las nóminas (índices), 
    se convierte a tipo object para permitir almacenar valores no numéricos, como la marca "x" """
Principal = Principal.astype(object)
for nombre, valor in nominas:
    if valor in Principal.index and nombre in Principal.columns:
        #Para cada par [Tema, Nómina] detectado previamente, se marca con "x" la celda correspondiente en la matriz Principal para indicar que el
        #trabajador identificado por la nómina requiere capacitación en ese tema.
        Principal.loc[valor, nombre] = "x"


In [217]:
Principal.to_csv('DNC2025terminado.csv', sep=',')

In [219]:
Principal

,metodologias_agiles_kanban,automatizacion_sistemas,ciberseguridad_basica,ciberseguridad_avanzada,desarrollo_web_frontend,desarrollo_web_backend,bases_de_datos_sql,bases_de_datos_nosql,devops_practicas,cloud_computing,...,container_orchestration,api_gateway_management,data_observability,predictive_maintenance_iiot,graph_ai,conversational_ai,explainable_ai_techniques,cloud_cost_optimization,zero_trust_architecture,privacy_preserving_ai
nominas,,,,,,,,,,,,,,,,,,,,,
776532,NaN,NaN,NaN,x,NaN,NaN,x,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
770479,NaN,NaN,NaN,x,NaN,NaN,x,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
763354,NaN,NaN,NaN,x,NaN,x,x,NaN,NaN,x,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
775239,NaN,NaN,NaN,x,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
861202,NaN,NaN,NaN,x,NaN,NaN,x,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
762375,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,x,...,x,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
761244,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,x,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
772870,NaN,NaN,NaN,NaN,NaN,NaN,x,NaN,NaN,x,...,x,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
